<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/master/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Striving for Simplicity: The All Convolutional Net
###### *Wei-Tse Yang, Sunwei Wang, and Qingyuan Cao*
###### 14/4/2020
In this notebook, we try to reproduce the TABLE 3 in [original paper](https://arxiv.org/abs/1412.6806). The source code of one of the models on ```Pytorch``` and the training procedure on Google Colab can be found in [github](https://github.com/StefOe/all-conv-pytorch). **We adopt the original training procedure and change it to the Python class. Also, we build the models from scratch on Pytorch.**


---
# Brief Introduction
The paper shows that replacing the maxpooling with the convolutional with increased strides can improve the performance. The authors first conduct four kinds of CNN models. **"Model"** is CNN with max-pooling. **"Strided-CNN"** removes max-pooling. **"All-CNN" ** replaces the max-pooling with a convolutional layer with strides. **"ConvPool-CNN" ** has the max-pooling and one-more convolutional layer before the pooling. The results show that ....



---
# Experiment Setup 

---
# Results 
A Table


In [0]:
# Copy the trianing chunk to here?

# Cross-Valudation 

In [0]:
# Copy the trianing chunk to here?

# DropOut and Batch Normalization
Add motivation and description. 

In [0]:
# Copy the trianing chunk to here?

# Optimizer
Add motivation and description. 

In [0]:
# Copy the trianing chunk to here?

# Softmax?

In [0]:
# Copy the trianing chunk to here?

# Appendix


*   [Github Repository](https://github.com/wtsyang/dl-reproducibility-project)
*   [A Python Class to build all Models](https://github.com/wtsyang/dl-reproducibility-project/blob/master/Model.py)
*   [A Python Class for Training Procedure](https://github.com/wtsyang/dl-reproducibility-project/blob/master/Training.py)
*   [The Notebook for Model A]()







